In [1]:
import os
import re
import string
from functools import partial

import nltk
import polars as pl
import tensorflow as tf
from nltk.corpus import stopwords

nltk.download('stopwords')

2023-08-06 23:29:43.549128: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-06 23:29:43.581176: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-06 23:29:43.581812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 23:29:44.313691: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
device = tf.config.list_physical_devices()
device

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
file = 'dev.gzip'
dev_df = pl.read_parquet(file)
dev_df.head()

Log,Target
str,str
""" 1119803499 20…","""normal"""
""" 1119803105 20…","""normal"""
""" 1121496169 20…","""normal"""
""" 1120968564 20…","""normal"""
""" 1120953205 20…","""normal"""


In [4]:
from utils.utils import get_dataset, get_tokenizer

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
dev_dataset = get_dataset(file_path= file, batch_size=1,shuffle= False)

In [6]:
for word in dev_dataset.take(2):
    print(word)

(<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'rmnccju rmnccju ras kernel info data store interrupt caused dcbf'],
      dtype=object)>, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'rmnecju rmnecju ras kernel info program interrupt illegal instruction'],
      dtype=object)>, <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>)


In [7]:
token_vec, voc = get_tokenizer(dev_dataset, sequence_length= 15)

In [8]:
out = token_vec(['rmnccju rmnccju ras kernel info data store interrupt caused dcbf'])
out

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 26,  26,   3,   4,   5,  35,  75,  25,  95, 242,   0,   0,   0,
          0,   0]])>

In [9]:
log_ds = dev_dataset.map(lambda text,label: text)
sequence_length = 10
tokenizer_layer = tf.keras.layers.TextVectorization(split= 'whitespace', output_mode= 'int',
                                              output_sequence_length= sequence_length)
tokenizer_layer.adapt(log_ds)
vocab_size = tokenizer_layer.vocabulary_size()